# Control-law design for full-state feedback

The purpose of the control law is to allow us to assign as et of pole locations for the close-loop system that will correspond to satisfactory dynamic response in term of rise time and other measures of transient response. Then it will be show how introducing a reference term in a full state feedback and finally the process of finding "good" poles for the design.

## Finding the control law

How does it change the state equation when the system is inside a control loop? Below it's show the figure of a design block scheme:

|          |         |
|----------|:--------|
|![control law\label{controllaw}](./images/control-law.png)|<p style="text-alignment=left;width:300px;"><a name="fig1.1">Fig. 1.1</a> - Assumed system for control law design</p>|

$$
u = -\pmb{Kx} = \begin{bmatrix}K_1 & K_2 & ... K_n \end{bmatrix} \label{eq:K} \tag{1.1}
$$

Equation $\eqref{eq:K}$ tell us that the system has a constant atrix in the state-vector feedback path gains, $K1,K2,....K_n$, and because there are n roots of the system, it's possible that there are enough degree of freedom to select arbitrarily and desired root location by choosing the proper value of $K_i$. Substituting the feedback law given in $\eqref{eq:K}$ into the system described by $\eqref{eq:stat-form}$ yields:  

$$
\dot{\pmb{x}} = \pmb{Fx - GKx} = (\pmb{F - GK}) \pmb x
$$

Now writing:
$$ 
\dot{\pmb x}(t) = p_i e^{p_i t} \pmb x_0 = (\pmb{F-GK}) \pmb x = (\pmb{F-GK}) e^{p_i t} \pmb x_0
$$
$$
\pmb x(0) = \pmb x_0
$$

or

$$
(\pmb{F-GK}) \pmb x_0 = p_i \pmb{x}_0
$$

This is an eigenvalues problem that it can be solved if and only if 

$$ det[s \pmb I - (\pmb{F-GK})] = 0 \label{eq:det} \tag{1.2}$$

When evaluate, this yield an nth order polynomial in s containing the gains **K** so the roots of $\eqref{eq:det}$ are in the *desirable* locations. We assume that desired locations are known, say,

$$s=s_1,s2,....,s_n$$

Then the corresponding desired (control) characteristic equation is

$$
\alpha_c(s) = (s-s_1)(s-s2)....(s-s_n) = 0 \label{eq:alpha} \tag{1.3}
$$

Hence the required elements of **K** are obtained by matching coefficients $\eqref{eq:det}$ and $\eqref{eq:alpha}$ ,forcing the systems's characteristic equation to be identical to the desired characteristic equation and the closed loop poles at the desired locations.

Calculating the gains by using this technique becomes rather tedious when the order of the system is higher than 3. There are another technique that use the control canonical form to simplify the matrix and finishing with the match between the above equations. However there is an alternative to this method called **Ackerman's formula** divided in three steps process of converting to $(\pmb F_c \pmb G_c)$ solving for the gains, and converting back again into the very compact form:  

$$
\pmb K = [0,0,.....0,1]{\mathcal{C}}^{-1} \alpha_c{\pmb F}
$$

where $\mathcal C$ is the controllability matrix in $\eqref{eq:C}$, n gives the order of the system and the number of the state variables and : 

$$
\alpha_c (\pmb F) = \pmb F^n + \alpha_1 \pmb F^{n-1} + ..... + \alpha_n \pmb I
$$

where the $\alpha_i$ are the coefficient corresponding to the desired poles locations.

As an example it's simulated a control-law with different pole placement to show how the cart pendulum change the overshoot and robustness. 
```
clear all, close all, clc

m = 1;
M = 5;
L = 2;
g = -10;
d = 1;

s = 1; % pendulum up (s=1)

A = [0 1 0 0;
    0 -d/M -m*g/M 0;
    0 0 0 1;
    0 -s*d/(M*L) -s*(m+M)*g/(M*L) 0];

B = [0; 1/M; 0; s*1/(M*L)];
eig(A)

rank(ctrb(A,B))  % is it controllable

%%  Pole placement
% For more complex case a more reliable formula is available with 
% function place
% p is a vector of desired eigenvalues
p0 = [-.01; -.02; -.03; -.04]; % not enough
p1 = [-.3; -.4; -.5; -.6];  % just barely
p2 = [-1; -1.1; -1.2; -1.3]; % good
p3 = [-2; -2.1; -2.2; -2.3]; % aggressive
p4 = [-3; -3.1; -3.2; -3.3]; % aggressive
%p = [-3.5; -3.6; -3.7; -3.8]; % breaks
% create a list of vectors
list_p = {p1,p2,p3,p4};
% K = lqr(A,B,Q,R);

% Plot a list of non-minimum phase zero 
for k = 1:numel(list_p)
    tspan = 0:.001:30;
    vec_p = list_p{k};
    % Used place instead acker function
    K = place(A,B,vec_p);
    y0 = [-3; 0; pi+.1; 0];
    [t,y] = ode45(@(t,y)cartpend(y,m,M,L,g,d,-K*(y-[1; 0; pi; 0])),tspan,y0);    
    plot(t,y(:,1)); hold on;
end
```

|          |         |
|----------|:--------|
|![modalblock\label{modalblock}](./images/poleplacement.png)|<p style="text-alignment=left;width:300px;"><a name="fig1.2">Fig. 1.2</a> - Some $K$ values based on different pole placement. The start position is x = -3 and $\theta= \pi +0.1$. Finale reference position is x = 1 $\theta = \pi$. It's worthwhile to note than more aggressively is equal to require more energy and less robustness </p>|